# Creating NEEM Data

In order to create neems we need to load two packages, the plan that is to be executed and the cram-cloud logger that provides the neem logging capabilities. Loading the package can be done with (ros-load:load-system "ros package name" :asdf-package-name)

First we load the package containing the pick and place plan for the pr2 with ```ros-load:load-system```. For this example we are using "cram_pr2_pick_place_demo". 

Remember that the asd file name is the same as the ros package name, only that underscores are replaced with hyphons 

Important: Always wait till the commands finish and return the solutions.

In [ ]:
(ros-load:load-system "cram_pr2_pick_place_demo" :cram-pr2-pick-place-demo)

Then we load the cram cloud logger (ccl) with "cram_cloud_logger"

In [ ]:
(ros-load:load-system "cram_cloud_logger" :cram-cloud-logger)

The ccl uses the predicates defined in the neem-interface.pl. These predicates provide the ability to start/stop episodes and actions as well as add parameters to actions such as participants. These predicates are not standart to knowrob and have to be loaded before the start of the logging.


If knowrob runs on the same PC as cram no extra steps have to be taken since CCL comes with the neem-interface.pl. Because we use container, we have to tell knowrob where to finde the neem-interface. CCL can do this by setting the neem-interface-path. You can use ```setf``` to set the value of a variable in lisp. The variable pointing to the neem interface is ```ccl::*neem-interface-path*```. In the docker container the neem is saved in ```'/home/ros/src/neem-interface/src/neem-interface.pl'```

In [ ]:
 (setf ccl::*neem-interface-path* "'/home/ros/src/neem-interface/src/neem-interface.pl'")

Neems contain subsymbolic data such as TF. In projection we have to set the topic to which TF is published ("tf") and enable the publishing. The respective variables are ```cram-tf:*tf-broadcasting-topic*``` and ```cram-tf:*tf-broadcasting-enabled*```.

In [ ]:
(setf cram-tf:*tf-broadcasting-topic* "tf")

In [ ]:
(setf cram-tf:*tf-broadcasting-enabled* t)

Now we connect our environment to ROS with ```roslisp-utilities:startup-ros```

In [ ]:
 (roslisp-utilities:startup-ros :name "cram" :anonymous nil)

Now comes the actual part of recording neems. The recording is initialized by the funktion ```ccl:start-episode```.

In [ ]:
 (ccl:start-episode)

Now that the recording is started, we can execute the plan. One example is to execute the setting-demo using the simulated robot.

In [ ]:
(urdf-proj:with-simulated-robot
 (demo::setting-demo '(:bowl)))

When the plan is finished we have to stop the neem recording with ```ccl:stop-episode```. This will save the NEEM where the environment variable KNOWROB_MEMORY_DIR is pointing to. In case of the docker container it is /home/neem_data

In [ ]:
 (ccl:stop-episode)